<a href="https://colab.research.google.com/github/OrrangeMoon/my_script/blob/main/stock_pprice_curve_and_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# READ


In [4]:
################################################################################


import csv
# from BaseHTTPServer import BaseHTTPRequestHandler,HTTPServer
import http.server
import json
import operator
import os.path
import re
import threading
from datetime import timedelta, datetime
# from itertools import izip
from random import normalvariate, random
from socketserver import ThreadingMixIn

import dateutil.parser

################################################################################
#







# Config

In [5]:
# Config

# Sim params

REALTIME = True
SIM_LENGTH = timedelta(days=365 * 5)
MARKET_OPEN = datetime.today().replace(hour=0, minute=30, second=0)

# Market parms
#       min  / max  / std
#SPD = (2.0, 6.0, 0.1)
PX = (60.0, 150.0, 1)
FREQ = (12, 36, 50)

# Trades

OVERLAP = 4

# Test Data

In [6]:
################################################################################
#
# Test Data

def bwalk(min_val, max_val, std):
    """
    simulate the random walk of stock price(or anything fluctuate in certain range),return a generator of cur_val
    """
    avg=(max_val+min_val)/2
    while True:
        if min_val <= avg+normalvariate(0, std) <= max_val:
            yield normalvariate(0, std)+avg

# def market(t0=MARKET_OPEN):
#     """ Generates a random series of market conditions,
#         (time, price, spread).
#         t0: present time
#     """
#     for hours, px in zip(bwalk(*FREQ), bwalk(*PX)):
#         t0 += timedelta(hours=abs(hours))
#         spd=SPD(px=next(bwalk(*PX)),t0)
#         yield t0, px, spd
def market(t0=MARKET_OPEN,px0=next(bwalk(*PX))):
    for t,px in zip(bwalk(*FREQ),bwalk(*PX)):
        t_d=timedelta(hours=abs(t))
        t0+=t_d
        std=px-px0
        # print(f'当前时间：{t}')
        # print(f'时间过去了{t_d}小时')
        # print(f'原价为{px0}元')
        # print(f'当前价格为{px}元')
        # print(f'价格范围是{std}')
        yield t0,px,std
        t0,px0=t0,px

def orders(hist):
    """ Generates a random set of limit orders (time, side, price, size) from
        a series of market conditions.
        side: 'buy' or 'sell',
        stock: 'ABC' or 'DEF',
        hist:market conditions,
        d:+2(sell) or -2(buy),
        mu:px + (spd / d)
        sigma:spd / OVERLAP
        order:the trade price
    """
    for t, px, spd in hist:
        stock = 'ABC' if random() > 0.5 else 'DEF'
        #randomly choose buy or sell
        side, d = ('sell', 2) if random() > 0.5 else ('buy', -2)
        #generate random number and round to integer, 2 means keep two digits after decimal
        order = round(normalvariate(px + (spd / d), spd / OVERLAP), 2)
        #the number of stock which has been traded
        size = int(abs(normalvariate(0, 100)))
        yield t, stock, side, order, size


# CSV


In [35]:
import math
def generate_csv(flag=-1):#flag的取值为股票订单的数量,flag>-1时,生成flag条，默认flag=50;flag=-1时，不断生成无限条直到超出时间限制
    """ Generate a CSV of order history. """
    if flag==-1:flag=math.inf

    with open('test.csv', 'w',encoding='UTF-8') as f:
        writer = csv.writer(f)
        writer.writerow(['t', 'stock', 'side', 'order', 'size'])
        n=0
        for t, stock, side, order, size in orders(market()):
          n+=1
          if float(n)==float(flag):break
          if t > MARKET_OPEN + SIM_LENGTH:
            break
          writer.writerow([t, stock, side, order, size])


def read_csv():
    """ Read a CSV or order history into a list. """
    with open('test.csv', 'r') as f:
        reader=csv.reader(f)
        for time, stock, side, order, size in reader:
          try:
            yield dateutil.parser.parse(time), stock, side, float(order), int(size)
          except dateutil.parser.ParserError:pass




In [36]:
generate_csv(50)
for i in read_csv():
  print(i)


(datetime.datetime(2024, 4, 15, 19, 34, 48, 177510), 'DEF', 'sell', 108.26, 37)
(datetime.datetime(2024, 4, 17, 2, 41, 52, 750211), 'ABC', 'buy', 105.63, 10)
(datetime.datetime(2024, 4, 17, 15, 39, 0, 967978), 'DEF', 'sell', 103.5, 88)
(datetime.datetime(2024, 4, 18, 6, 42, 24, 907693), 'DEF', 'buy', 104.81, 112)
(datetime.datetime(2024, 4, 22, 19, 51, 26, 833646), 'DEF', 'sell', 103.14, 81)
(datetime.datetime(2024, 4, 23, 12, 49, 46, 878620), 'ABC', 'sell', 105.23, 6)
(datetime.datetime(2024, 4, 23, 23, 1, 46, 87253), 'DEF', 'sell', 108.45, 128)
(datetime.datetime(2024, 4, 26, 20, 8, 45, 377218), 'ABC', 'sell', 105.28, 32)
(datetime.datetime(2024, 4, 27, 16, 8, 54, 403743), 'ABC', 'sell', 105.15, 147)
(datetime.datetime(2024, 4, 28, 23, 41, 52, 567791), 'ABC', 'buy', 105.36, 74)
(datetime.datetime(2024, 4, 30, 8, 25, 42, 955529), 'DEF', 'sell', 102.05, 2)
(datetime.datetime(2024, 4, 30, 9, 35, 3, 452744), 'ABC', 'buy', 104.24, 241)
(datetime.datetime(2024, 5, 1, 14, 9, 57, 441348), 'D